In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from matplotlib.colors import ListedColormap
from sklearn import metrics
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import RobustScaler

In [2]:
train=pd.read_csv("Train_swa.csv")
train.head()

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa


In [3]:
train["content"][0]

' SERIKALI imesema haitakuwa tayari kuona amani na utulivu wa nchi inachezewa huku ikisisitiza uwepo wa umoja kati ya wananchi bila kujali tofauti ya imani, kabila au itikadi yoyote.Hayo yalisemwa na Naibu Waziri wa Mambo ya Ndani ya Nchi, Hamad Yussuf Masauni wakati akifungua semina ya siku mbili iliyofanyika jijini Dar es Salaam ikiwahusisha viongozi wa taasisi za Kiislamu, lengo ikiwa ni kuwakumbusha kuhubiri amani katika sehemu zao.Naibu Waziri amesema mwelekeo na malengo ya Serikali ya Awamu ya Tano ni kukuza maendeleo katika sehemu mbalimbali nchini lengo ikiwa kuinua maisha ya wananchi na nchi kwa ujumla.“Serikali hii imejidhatiti kuhakikisha maendeleo yanakuja kwa kasi na maendeleo hayawezi kuja ikiwa amani na utulivu haupo, sisi kama serikali tutahakikisha tunalinda amani iliyopo ili wananchi wapate kufanya shughuli za kiuchumi bila wasiwasi wowote,” amesema Masauni.Akizungumza wakati wa ufunguzi huo, Shehe wa Mkoa wa Dar es Salaam, Alhaji Alhad Mussa Salum aliihakikishia seri

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5151 non-null   object
 1   content   5151 non-null   object
 2   category  5151 non-null   object
dtypes: object(3)
memory usage: 120.9+ KB


In [5]:
train["category"].nunique()

5

In [6]:
train["category"].unique()

array(['Kitaifa', 'Biashara', 'michezo', 'Kimataifa', 'Burudani'],
      dtype=object)

In [7]:
#text preprocessing
import nltk

In [8]:
import string,re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
#defining swahili stopwords
swahili_stopwords = set([
    "na", "ni", "ya", "kwa", "wa", "katika", "la", "kama", "hiyo", "hili",
    "huyo", "yake", "hata", "kuwa", "tu", "lakini", "sana", "pia", "ndiyo",
    "au", "bali", "huku", "hao", "mimi", "yangu", "yake", "yetu"
])


In [10]:
def preprocess_content(content):
    # Convert to lowercase & tokenize
    tokens = nltk.word_tokenize(content.lower())

    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    # Remove stopwords
    tokens = [word for word in tokens if word not in swahili_stopwords]

    # Join tokens back into a sentence
    processed_text = " ".join(tokens)

    return processed_text


In [11]:
#reading test
test= pd.read_csv("Test_swa.csv")
test.head()

,swahili_id,content
0,ae3baa6c34aa523fd2aa4de3c89448efff922311,Rais John Magufuli amemuagiza Msajili wa Hazi...
1,c4ee26a3ade8064a2ec494996e836900fd32dd8e,TAHARUKI imezuka katika mkutano wa Naibu Wazi...
2,58aee3aa1d94554ff57e6a053dbd60658e4890ff,"KOCHA wa Azam FC ya Dar es Salaam, Idd Cheche..."
3,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...
4,c83e9738ae5d1790ee85b99863deb734e7614c52,"WAZIRI wa Nchi, Ofi si ya Makamu wa Rais, Muu..."


In [71]:
test["content"][3]

' THAMANI ya mauzo ya bidhaa za Afrika Mashariki katika Umoja wa Ulaya (EU) yameongezeka hadi Dola za Marekani bilioni 2.5 (Sh trilioni 5.2 za Tanzania) mwaka 2017, sawa na ongezeko la asilimia nane.Taarifa hiyo ni kwa mujibu wa Kituo cha Biashara cha Kimataifa (ITC).Kwa mujibu wa ITC, bidhaa kutoka ukanda huo zilizouzwa Ulaya kwa mwaka 2016 zilikuwa na thamani ya Dola milioni 2.3 (Sh trilioni 4.83 za Tanzania), hivyo kumekuwa na ongezeko la takribani Sh bilioni 400.\xa0Mkurugenzi wa Sekta ya Uzalishaji ya EAC, Jean Baptiste, amesema hivi karibuni jijini Arusha yalipo makao makuu ya jumuiya hiyo kuwa, ili kuliteka zaidi soko la EU, ukanda wa Afrika Mashariki unapaswa kuelekeza nguvu katika ubora utakaokwenda sambamba na ule wa Ulaya.Miongoni mwa bidhaa za EAC zenye soko katika jumuiya ya Ulaya ni pamoja na kahawa, maua, chai, pamba, samaki na mboga.Mkurugenzi Mtendaji wa Baraza la Biashara Afrika Mashariki (EABC), Lillian Awinja, alisema suala la maridhiano katika biashara baina ya EU 

In [13]:
#joining train and test sets for preprocessing 
df3 = pd.concat([train[['content']], test[['content']]])
df3.head() 

,content
0,SERIKALI imesema haitakuwa tayari kuona amani...
1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti..."
2,SERIKALI imetoa miezi sita kwa taasisi zote z...
3,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...
4,WATANZANIA wamekumbushwa kusherehekea sikukuu...


In [14]:
#applying preprocessing in the content column 
df3['processed_content'] = df3['content'].apply(preprocess_content)
df3.tail(10)

,content,processed_content
1020,WIKI iliyopita tulizungumzia umuhimu wa kuing...,wiki iliyopita tulizungumzia umuhimu kuingiza ...
1021,HUZUNI imetanda baada ya fi limbi ya mwisho y...,huzuni imetanda baada fi limbi mwisho mwamuzi ...
1022,KOCHA mkuu wa timu ya soka ya Mbao FC ya mkoa...,kocha mkuu timu soka mbao fc mkoani mwanza amr...
1023,Waziri wa Ardhi Nyumba na Maendeleo ya Makazi...,waziri ardhi nyumba maendeleo makazi william l...
1024,HALMASHAURI ya Manispaa ya Singida imepanga k...,halmashauri manispaa singida imepanga kufunga ...
1025,"MKURUGENZI wa Jiji la Arusha, Dk Maulid Maden...",mkurugenzi jiji arusha dk maulid madeni amesem...
1026,Hayo yalisemwa na Katibu Mkuu Wizara ya Afrik...,hayo yalisemwa katibu mkuu wizara afrika masha...
1027,Alitoa kauli hiyo juzi wakati akizungumza na ...,alitoa kauli juzi wakati akizungumza viongozi ...
1028,SIMBA imetinga raundi ya nne ya 32 bora ya Ko...,simba imetinga raundi nne 32 bora kombe shirik...
1029,"KOCHA wa Azam FC, Mholanzi Hans van der Pluij...",kocha azam fc mholanzi hans van der pluijm ame...


In [15]:
#vectorization of processed content 
vectorizer = TfidfVectorizer()
x=vectorizer.fit_transform(df3["processed_content"])
x.shape

(6181, 82234)

In [16]:
size_train = len(train)
size_test = len(test)

# Split back to train nd test
x_train = x[:size_train]  # First part for train
x_test = x[:size_test]  # Second part for test
print(x_train.shape, x_test.shape)

(5151, 82234) (1030, 82234)


In [58]:
y_train=train["category"]
y_train.shape

(5151,)

In [59]:
pd.DataFrame(y_train).nunique()

category    5
dtype: int64

In [60]:
#encoding target column 
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
pd.DataFrame(y_train).nunique()

0    5
dtype: int64

In [81]:
model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, tree_method='hist')
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [83]:
pred=model.predict(x_test)
pd.DataFrame(pred)

,0
0,3
1,0
2,3
3,4
4,3
...,...
1025,4
1026,3
1027,0
1028,4


In [85]:
print("Label Mapping:", dict(enumerate(encoder.classes_)))

Label Mapping: {0: 'Biashara', 1: 'Burudani', 2: 'Kimataifa', 3: 'Kitaifa', 4: 'michezo'}


In [86]:
class_labels =encoder.classes_
class_labels

array(['Biashara', 'Burudani', 'Kimataifa', 'Kitaifa', 'michezo'],
      dtype=object)

In [87]:
onehot_encoder = OneHotEncoder(sparse_output=False)
binary_matrix = onehot_encoder.fit_transform(pd.DataFrame(pred))
binary_matrix.shape 

(1030, 5)

In [88]:
df_binary = pd.DataFrame(binary_matrix, columns=class_labels)
df_binary

,Biashara,Burudani,Kimataifa,Kitaifa,michezo
0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
1025,0.0,0.0,0.0,0.0,1.0
1026,0.0,0.0,0.0,1.0,0.0
1027,1.0,0.0,0.0,0.0,0.0
1028,0.0,0.0,0.0,0.0,1.0


In [89]:
pd.DataFrame(pred).nunique()

0    5
dtype: int64

In [90]:
new_order=['Kitaifa','michezo','Biashara','Kimataifa','Burudani']

In [91]:
df_binary=df_binary[new_order]
df_binary.head()

,Kitaifa,michezo,Biashara,Kimataifa,Burudani
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0


In [92]:
df_binary=df_binary.set_index(test["swahili_id"])
df_binary.head()

,Kitaifa,michezo,Biashara,Kimataifa,Burudani
swahili_id,,,,,
ae3baa6c34aa523fd2aa4de3c89448efff922311,1.0,0.0,0.0,0.0,0.0
c4ee26a3ade8064a2ec494996e836900fd32dd8e,0.0,0.0,1.0,0.0,0.0
58aee3aa1d94554ff57e6a053dbd60658e4890ff,1.0,0.0,0.0,0.0,0.0
00579c2307b5c11003d21c40c3ecff5e922c3fd8,0.0,1.0,0.0,0.0,0.0
c83e9738ae5d1790ee85b99863deb734e7614c52,1.0,0.0,0.0,0.0,0.0


In [93]:
df_binary.to_csv('submission_swa1.csv', index=True)

***log loss 24.59071458***